进入 21 世纪，生命科学特别是基因科技已经广泛而且深刻影响到每个人的健康生活，于此同时，科学家们借助基因科技史无前例的用一种全新的视角解读生命和探究疾病本质。人工智能（AI）能够处理分析海量医疗健康数据，通过认知分析获取洞察，服务于政府、健康医疗机构、制药企业及患者，实现个性化，可以循证的智慧医疗，推动创新，实现价值。

心血管病、糖尿病等慢性疾病，每年导致的死亡人数占总死亡人数的 80%，每年用于慢病医疗费用占中国公共医疗卫生支出的比例超过 13%。作为一种常见慢性疾病，糖尿病目前无法根治，但却能通过科学有效的干预、预防和治疗，来降低发病率和提高患者的生活质量。阿里云联合青梧桐健康科技有限公司主办天池精准医疗大赛——人工智能辅助糖尿病遗传风险预测，希望用人工智能的方法和思想处理、分析、解读和应用糖尿病相关大数据，让参赛选手设计高精度，高效，且解释性强的算法来挑战糖尿病精准预测这一科学难题，为学术界和精准医疗提供有力的技术支撑，帮助我们攻克糖尿病。


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt

plt.rcParams["font.sans-serif"] = ["Heiti TC"]  # 用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False  # 用来正常显示负号

train_data = pd.read_csv("./data/d_train_20180102.csv", encoding="utf-8")
test_data = pd.read_csv("./data/d_test_A_20180102.csv", encoding="utf-8")
test_data["血糖"] = pd.read_csv("./data/d_answer_a_20180128.csv", encoding="utf-8")
data = pd.concat([train_data, test_data], ignore_index=True)
data.head()

# 数据概况


整体数据集的大概分布情况


In [ ]:
data.describe()

初始数据集的年龄分布直方图


In [ ]:
fig, ax = plt.subplots(figsize=(16, 9), dpi=400)

n, bins_num, patches = ax.hist(data["年龄"], bins=10, alpha=0.75, edgecolor="white")

ax.plot(bins_num[:10], n, marker="o", color="yellowgreen", linestyle="--")

for i in range(len(n)):
    plt.text(
        x=bins_num[i] + (bins_num[1] - bins_num[0]) / 2,
        y=n[i] * 1.01,
        s=str(int(n[i])),
        ha="center",
        va="bottom",
        fontsize=20,
    )

print("频数:{}\n区间:{}".format(n, bins_num))

min_age = data["年龄"].min()
max_age = data["年龄"].max()

t1 = np.linspace(min_age, max_age, num=11)

plt.xticks(t1)
plt.tick_params(labelsize=18)
plt.xlabel("年龄区间（分组）", fontdict={"weight": "normal", "size": 20})
plt.ylabel("频数", fontdict={"weight": "normal", "size": 20})
plt.title("初始数据集的年龄分布——直方图", fontdict={"weight": "normal", "size": 20})

初始数据集的血糖分布图


In [ ]:
plt.figure(figsize=(16, 9), dpi=400)

# 设置图形的总体风格
sns.set_theme(style="whitegrid")

# 创建分布图
displot = sns.displot(
    data,
    x="血糖",
    kde=False,
    stat="density",
    kind="hist",
    bins=100,
    color="blue",
    edgecolor="black",
)

# 设置图形的标题和轴标签
displot.set(title="Distribution of Variable", xlabel="Value", ylabel="Frequency")

# 调整子图间距和边缘间距
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9)

(mu, sigma) = norm.fit(data["血糖"])  # mu均值  sigma方差

print("\n mu = {:.3f} and sigma = {:.3f}\n".format(mu, sigma))

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, sigma)
plt.plot(x, p, "k", linewidth=1)

plt.legend(
    [r"Normal distribution ($\mu={:.3f}, \sigma={:.3f}$)".format(mu, sigma)],
    loc="best",
    prop={"size": 10},
)

plt.tick_params(labelsize=15)
plt.show()

初始数据集的数据缺失情况


In [ ]:
# 统计每一列缺失值的百分比
missing_rate = (data.isnull().sum() / data.shape[0] * 100).sort_values(ascending=False)
# 将结果转换为dataframe
missing_rate_df = pd.DataFrame(missing_rate, columns=["Missing_Ratio(%)"])
missing_rate_df["Missing_Ratio(%)"] = missing_rate_df["Missing_Ratio(%)"].round(4)
# 重置索引，将列名作为一列
missing_rate_df = missing_rate_df.reset_index().rename(columns={"index": "col_name"})
missing_rate_df

In [ ]:
import random

plt.subplots(figsize=(18, 10), dpi=400)

plt.barh(
    missing_rate_df["col_name"],
    missing_rate_df["Missing_Ratio(%)"],
    color=[
        f"#{random.randint(0, 255):02x}{random.randint(0, 255):02x}{random.randint(0, 255):02x}"
        for _ in range(len(data))
    ],
)

for i, v in enumerate(missing_rate_df["Missing_Ratio(%)"]):
    plt.text(v + 0.1, i, str(v), va="center")

plt.xlabel("缺失比率")

年龄转换成分类数值变量


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(data["性别"])
label = le.transform(data["性别"])  # transform接口调取结果
print(le.classes_)  # 属性.classes_查看标签中究竟有多少类别

data = data.drop(data[data["性别"] == "??"].index)

le = LabelEncoder()
le = le.fit(data["性别"])
label = le.transform(data["性别"])  # transform接口调取结果
print(le.classes_)  # 属性.classes_查看标签中究竟有多少类别

data["性别"] = LabelEncoder().fit_transform(data.loc[:, "性别"])

有一条性别数据为未知，删除后再进行分类，女标签为 0 ，男标签为 1


- drop id 与 体检日期 两列数据【与预测值没有关系】


In [ ]:
train_data = train_data.drop(columns=["id", "体检日期"], axis=1)

test_data = test_data.drop(columns=["id", "体检日期"], axis=1)

data = data.drop(columns=["id", "体检日期"], axis=1)

- 乙肝两对半的五项指标，缺失值太多，相关性很低，考虑丢弃


In [ ]:
train_data = train_data.drop(
    columns=["乙肝表面抗原", "乙肝表面抗体", "乙肝e抗原", "乙肝e抗体", "乙肝核心抗体"],
    axis=1,
)

test_data = test_data.drop(
    columns=["乙肝表面抗原", "乙肝表面抗体", "乙肝e抗原", "乙肝e抗体", "乙肝核心抗体"],
    axis=1,
)

data = data.drop(
    columns=["乙肝表面抗原", "乙肝表面抗体", "乙肝e抗原", "乙肝e抗体", "乙肝核心抗体"],
    axis=1,
)

连续变量，诸多测量变量的箱型图，排除了'性别','年龄','乙肝表面抗原','乙肝表面抗体', '乙肝 e 抗原', '乙肝 e 抗体', '乙肝核心抗体'

- 性别 为 分类变量，不适用箱型图
- 年龄 不是 测量变量
- '乙肝表面抗原','乙肝表面抗体', '乙肝 e 抗原', '乙肝 e 抗体', '乙肝核心抗体' 缺失情况太严重，不做箱型图


# 预处理


删除缺失值较多的行，这里使用 20%为界限  
其余 NAN 值使用该属性列的均值填充


In [ ]:
# 计算每一行缺失值的比率
missing_count = data.isna().sum(axis=1)
missing_ratio = missing_count / len(data.columns)

# 筛选需要删除的行
to_drop = missing_ratio[missing_ratio > 0.2].index
data.drop(to_drop, inplace=True)

# 用平均值填充剩余的缺失值
data.fillna(data.mean(), inplace=True)
print(data.shape)

再次查看数据矩阵的分布情况


In [ ]:
data.describe()

In [ ]:
plt.subplots(figsize=(15, 9), dpi=400)

ax = sns.boxenplot(
    data=data.loc[
        :,
        [
            "*天门冬氨酸氨基转换酶",
            "*丙氨酸氨基转换酶",
            "*碱性磷酸酶",
            "*r-谷氨酰基转换酶",
            "尿酸",
            "血红蛋白",
            "红细胞平均血红蛋白浓度",
            "血小板计数",
        ],
    ],
    outlier_prop=0.05,
    palette="Set2",
    linewidth=0.25,
)
plt.setp(ax.get_xticklabels(), rotation=15)
plt.tick_params(labelsize=12)

In [ ]:
plt.subplots(figsize=(15, 9), dpi=400)
cols = [
    i
    for i in data.columns
    if i
    not in [
        "*天门冬氨酸氨基转换酶",
        "*丙氨酸氨基转换酶",
        "*碱性磷酸酶",
        "*r-谷氨酰基转换酶",
        "尿酸",
        "血红蛋白",
        "红细胞平均血红蛋白浓度",
        "血小板计数",
        "性别",
        "血糖",
    ]
]
ax = sns.boxenplot(data[cols], outlier_prop=0.03, palette="Set2", linewidth=0.25)
plt.setp(ax.get_xticklabels(), rotation=65)
plt.tick_params(labelsize=12)

In [ ]:
data = data.drop(data[data["*天门冬氨酸氨基转换酶"] > 200].index)
data = data.drop(data[data["*丙氨酸氨基转换酶"] > 250].index)
data = data.drop(data[data["*r-谷氨酰基转换酶"] > 600].index)
data = data.drop(data[data["血小板计数"] > 600].index)
data = data.drop(data[data["*球蛋白"] > 60].index)
data = data.drop(data[data["甘油三酯"] > 30].index)
data = data.drop(data[data["肌酐"] > 175].index)
print(data.shape)

In [ ]:
# data.to_csv('drop_fillna.csv')

多重线性回归分析


In [ ]:
import statsmodels.api as sm

x = sm.add_constant(data.iloc[:, :-1])
y = data["血糖"]
model = sm.OLS(y, x).fit()  # 拟合模型
model.summary()

In [ ]:
# 以下是使用Pandas、NumPy和StatsModels库进行多重线性回归模型评估的代码：

# 计算均方误差（MSE）和决定系数（R²）：

import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 计算预测值
y_pred = model.predict(x)

# 计算MSE
mse = mean_squared_error(y, y_pred)

# 计算R²
r2 = r2_score(y, y_pred)

# 打印MSE和R²
print("MSE: ", mse)
print("R²: ", r2)
# 在上述代码中，我们首先使用predict()函数计算回归模型的预测值，然后使用mean_squared_error()函数计算均方误差，使用r2_score()函数计算决定系数，并打印出结果。

# 绘制残差图和QQ图：

# 计算残差
residuals = y - y_pred

# 绘制残差图
plt.scatter(y_pred, residuals)
plt.axhline(y=0, color="r", linestyle="-")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

# 绘制QQ图
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()
# 在上述代码中，我们首先计算残差，然后使用scatter()函数绘制残差图。我们还通过axhline()函数添加了一条水平线，以帮助我们评估残差是否随机分布。接下来，我们使用probplot()函数绘制QQ图，以评估残差是否服从正态分布。

数据标准化


In [ ]:
from sklearn.preprocessing import StandardScaler

dataset = data.iloc[:, :-1]
print(dataset.shape)
# print(dataset.isnull().sum())
target = data.iloc[:, -1]
print(target.shape)
# print(target.isnull().sum())

z_score = pd.DataFrame(StandardScaler().fit_transform(dataset), columns=dataset.columns)

z_score["性别"] = z_score["性别"].astype("int32")
z_score["年龄"] = z_score["年龄"].astype("int64")

z_score.info()

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.05)

# 训练模型
model.fit(z_score, target)

# 输出特征重要性
importances = pd.DataFrame({"feature": z_score.columns, "importance": abs(model.coef_)})
importances = importances.sort_values("importance", ascending=False).reset_index(
    drop=True
)
importances

热力图相关性分析


In [ ]:
mask = np.zeros_like(z_score.corr(), dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

fig_corr, ax_corr = plt.subplots(figsize=(28, 24), dpi=400)

cmap = sns.diverging_palette(250, 5, as_cmap=True)

# seaborn.diverging_palette(h_neg, h_pos, s=75, l=50, sep=1, n=6, center='light', as_cmap=False)
# Make a diverging palette between two HUSL colors.
# Parameters:
# h_neg, h_pos : float in [0, 359]
# Anchor hues for negative and positive extents of the map
# as_cmap : bool, optional
# If True, return a matplotlib.colors.ListedColormap

sns.heatmap(
    data=z_score.corr(),
    mask=mask,
    cmap=cmap,
    center=0,
    fmt=".2f",
    square=True,
    annot=True,
    linewidth=0.5,
    ax=ax_corr,
    annot_kws={"fontsize": 14},
)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
z_score = z_score.drop(
    columns=[
        "*天门冬氨酸氨基转换酶",
        "总胆固醇",
        "红细胞压积",
        "血红蛋白",
        "血小板比积",
        "红细胞平均血红蛋白量",
        "白球比例",
        "中性粒细胞%",
    ],
    axis=1,
)
print(z_score.shape)

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# Bartlett's球状检验
chi_square_value, p_value = calculate_bartlett_sphericity(z_score)
print("Bartlett球状检验卡方值:{}\nP值:{}".format(chi_square_value, p_value))
print()

# KMO检验
# 检查变量间的相关性和偏相关性，取值在0-1之间；KMO统计量越接近1，变量间的相关性越强，偏相关性越弱，因子分析的效果越好。
# 通常取值从0.6开始进行因子分析

kmo_all, kmo_model = calculate_kmo(z_score)
print("KMO检验参数\n", kmo_all)
print(kmo_model)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=26)
pca.fit(z_score)
variance = pca.explained_variance_ratio_
var = np.cumsum(np.round(variance, 3) * 100)
print(var)
plt.figure(figsize=(8, 4), dpi=400)
plt.ylabel("Variance Explained")
plt.xlabel("Number of Features")
plt.title("PCA Analysis")
plt.ylim(10, 110)
plt.plot(var)

In [ ]:
pca = PCA(n_components=0.98, svd_solver="full")
pca.fit(z_score)
variance = pca.explained_variance_ratio_
var = np.cumsum(np.round(variance, 3) * 100)
print(var)

column = [
    "component1",
    "component2",
    "component3",
    "component4",
    "component5",
    "component6",
    "component7",
    "component8",
    "component9",
    "component10",
    "component11",
    "component12",
    "component13",
    "component14",
    "component15",
    "component16",
    "component17",
    "component18",
    "component19",
    "component20",
    "component21",
    "component22",
    "component23",
]
pca_trans = pca.transform(z_score)
pca_df = pd.DataFrame(pca_trans, columns=column)
pca_df.head()

In [ ]:
# pca_df.to_csv('z_score_pca.csv')
# target.to_csv('target.csv')

训练集测试集划分


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("z_score_pca.csv", index_col=0)
target = pd.read_csv("target.csv", index_col=0)
print(df.shape)

train_features, test_features, train_labels, test_labels = train_test_split(
    df, target, test_size=0.3, random_state=23
)

train_labels = train_labels.values.ravel()
test_labels = test_labels.values.ravel()

print(train_features.shape, train_labels.shape)
print(test_features.shape, test_labels.shape)

多重线性回归


In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

LinearR = linear_model.LinearRegression()
model = LinearR.fit(train_features, train_labels)
prediction_train = LinearR.predict(train_features)
print("训练集的MSE为:{}".format(mean_squared_error(train_labels, prediction_train)))
print("训练集的R2为:{}".format(r2_score(train_labels, prediction_train)))
prediction_test = LinearR.predict(test_features)
print("测试集的MSE为:{}".format(mean_squared_error(test_labels, prediction_test)))
print("测试集的R2为:{}".format(r2_score(test_labels, prediction_test)))

# coef_ 系数w1,w2.....wn
print(LinearR.coef_)

# intercept_ 截距
print(LinearR.intercept_)

神经网络


In [ ]:
import tensorflow as tf  # 深度神经网络
from tensorflow import keras
from keras.layers import Dense, PReLU, Dropout, BatchNormalization

print(tf.__version__)
print(tf.keras.utils.custom_object_scope)

In [ ]:
model = tf.keras.Sequential(
    [  # 根据情况调整模型结构，Sequential模型，可进行层的堆叠
        Dense(units=400, input_dim=train_features.shape[1], activation=PReLU()),
        Dropout(0.4),
        Dense(units=160, kernel_initializer="normal", activation=PReLU()),
        BatchNormalization(),
        Dropout(0.6),
        Dense(units=64, kernel_initializer="normal", activation=PReLU()),
        BatchNormalization(),
        Dropout(0.5),
        Dense(units=26, kernel_initializer="normal", activation=PReLU()),
        BatchNormalization(),
        Dropout(0.6),
        Dense(1, kernel_initializer="normal"),
    ]
)

# Dense(units,activation=None,use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None)
# units：大于0的整数，代表该层的输出维度
# input_dim：该层输入的维度
# activation：激活函数，为预定义的激活函数名（参考激活函数），或逐元素（element-wise）的Theano函数。如果不指定该参数，将不会使用任何激活函数（即使用线性激活函数：a(x)=x）

# use_bias：布尔值，是否使用偏置项
# kernel_initializer：权值初始化方法，为预定义初始化方法名的字符串，或用于初始化权重的初始化器。
# bias_initializer：偏置向量初始化方法，为预定义初始化方法名的字符串，或用于初始化偏置向量的初始化器。
# regularizer：正则项，kernel为权重的、bias为偏执的，activity为输出的
# constraints：约束项，kernel为权重的，bias为偏执的

model.compile(loss="mse", optimizer="adamax")
# keras model.compile(loss='目标函数 ', optimizer='adam', metrics=['accuracy'])
# loss目标函数，或称损失函数，是网络中的性能函数（mse就是mean_squared_error均方差）
# optimizer优化器，用于控制梯度裁剪（sgd为随机梯度下降法，支持动量参数，支持学习衰减率，支持Nesterov动量）

model.summary()  # 打印出模型概况，实际调用的是keras.utils.print_summary

In [ ]:
myModel = model.fit(
    train_features,
    train_labels,
    epochs=100,
    batch_size=8,
    validation_data=(test_features, test_labels),
)
# train_features：输入数据。一个输入，类型是numpy.array
# train_labels：输出数据
# batch_size：整数，指定进行梯度下降时每个batch包含的样本数。训练时一个batch的样本会被计算一次梯度下降，使目标函数优化一步。
# epochs：整数，训练终止时的epoch值，训练将在达到该epoch值时停止，当没有设置initial_epoch时，它就是训练的总轮数，否则训练的总轮数为epochs - inital_epoch

In [ ]:
# print(myModel.history)
import matplotlib.pyplot as plt

epochs = range(len(myModel.history["loss"]))

plt.figure(dpi=400)
plt.plot(epochs, myModel.history["loss"], "b", label="Training loss")
plt.plot(epochs, myModel.history["val_loss"], "r", label="Validation val_loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("Traing and Validation loss")
plt.legend()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

test_preds = model.predict(test_features)  # 模型预测，输入测试集，输出预测结果
print("测试集MSE为:%.5f" % mean_squared_error(test_labels, test_preds))
print("测试集R^2:%.5f" % r2_score(test_labels, test_preds))

弱学习器 决策树 寻找最优参数组合（随机搜索）


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

DeTree = DecisionTreeRegressor(
    criterion="squared_error", random_state=30, splitter="random"
)

# 定义参数分布
param_dist = {
    "max_depth": [4, 5, 6, 7],
    "min_samples_leaf": np.arange(1, 30),
    "min_samples_split": np.arange(2, 30),
}

# 使用随机搜索寻找最优参数组合
random_search = RandomizedSearchCV(
    DeTree,
    param_distributions=param_dist,
    n_iter=3000,
    cv=5,
    scoring="neg_mean_squared_error",
)
random_search.fit(train_features, train_labels)

# 输出最优参数组合
print("随机搜索的最优参数组合：", random_search.best_estimator_)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

DeTree = DecisionTreeRegressor(
    criterion="squared_error",
    max_depth=5,
    min_samples_leaf=4,
    min_samples_split=14,
    random_state=30,
    splitter="random",
)

weakClassifier = DeTree.fit(train_features, train_labels)
y_pred = weakClassifier.predict(test_features)
print("测试集MSE值: %0.5f" % (mean_squared_error(test_labels, y_pred)))

AdaBoost


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# 创建Adaboost回归模型
adaboost = AdaBoostRegressor(estimator=weakClassifier, random_state=32)

# 定义参数分布
param_dist = {
    "n_estimators": [243, 244, 245, 246, 247, 248, 249, 250],
    "learning_rate": np.logspace(-3, 0, 4),
    "loss": ["linear", "square", "exponential"],
}

# 使用网格搜索寻找最优参数组合
grid_search = GridSearchCV(
    adaboost, param_grid=param_dist, cv=5, scoring="neg_mean_squared_error", n_jobs=4
)
grid_search.fit(train_features, train_labels)

# 输出最优参数组合
print("最优参数组合:", grid_search.best_params_)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score

adaboost = AdaBoostRegressor(
    estimator=weakClassifier,
    n_estimators=246,
    learning_rate=0.01,
    loss="exponential",
    random_state=32,
)

# base_estimator：基础回归模型，默认为DecisionTreeRegressor(max_depth=3)，也可以使用其他的回归模型。
# n_estimators：基础回归模型的数量，默认为50。
# learning_rate：学习率，默认为1.0。学习率控制每个基础回归模型的权重，在每个迭代中对误差进行调整，以提高模型性能和稳定性。
# loss：损失函数，默认为linear。可以选择linear、square或exponential。损失函数用于调整每个基础回归模型的权重。
# random_state：随机种子。
# 需要注意的是，Adaboost回归模型的参数n_estimators和learning_rate是相互影响的。当基础回归模型数量较多时，学习率应该设置较小以避免过拟合。反之，当基础回归模型数量较少时，学习率可以设置较大以提高模型的性能。

# 使用交叉验证评估模型精度
scores = cross_val_score(
    adaboost, train_features, train_labels, cv=5, scoring="neg_mean_squared_error"
)

# 计算交叉验证的平均MSE值
cv_mse = -scores.mean()
print("交叉验证MSE值: %0.5f" % cv_mse)

# 训练模型并预测测试集
adaboost.fit(train_features, train_labels)
y_pred = adaboost.predict(test_features)

# 计算测试集的MSE值
test_mse = mean_squared_error(test_labels, y_pred)
test_r2 = r2_score(test_labels, y_pred)
print("测试集MSE值: %0.5f" % test_mse)
print("测试集R^2值: %0.5f" % test_r2)

XGBoost


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# 创建XGBoost回归模型
xgb = XGBRegressor()

# 定义参数网格
param_grid = {
    "learning_rate": [0.01, 0.1, 1.0],
    "max_depth": [3, 5, 7],
    "min_child_weight": [1, 3, 5],
    "gamma": [0.0, 0.1, 0.2],
    "subsample": [0.5, 0.7, 0.9],
    "colsample_bytree": [0.3, 0.5, 0.7],
}

# 使用网格搜索寻找最优参数组合
grid_search = GridSearchCV(
    xgb, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=4
)
grid_search.fit(train_features, train_labels)

# 输出最优参数组合
print("最优参数组合:", grid_search.best_params_)

In [ ]:
import xgboost as xgb
import numpy as np

dtrain = xgb.DMatrix(train_features, label=train_labels)
dtest = xgb.DMatrix(test_features, label=test_labels)

# params:弱评估器(booster)有关的参数
params = {
    "colsample_bytree": 0.5,
    "gamma": 0.2,
    "learning_rate": 0.1,
    "max_depth": 3,
    "min_child_weight": 5,
    "subsample": 0.9,
}
num_round = np.arange(30, 51, 1)  # 迭代轮数

for i in num_round:
    boost = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=i,  # 设置模型参数  # dtrain：训练数据集
    )  # 迭代轮数
    result = boost.predict(dtest)
    print(
        "迭代{}次的均方误差MSE为:{}".format(i, mean_squared_error(test_labels, result))
    )
    print("迭代{}次的R^2为:{}".format(i, r2_score(test_labels, result)))

XGBoost 迭代最优：
迭代 40 次的均方误差 MSE 为:1.6605730332950106
迭代 40 次的 R^2 为:0.11773580292133157


RandomForest


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# # 定义模型
# model = RandomForestRegressor()

# # 定义参数空间
# param_dist = {
#     'n_estimators': range(50, 100, 5),
#     'max_depth': [None] + list(range(1, 14, 2)),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'min_samples_split': list(range(2, 11)),
#     'min_samples_leaf': list(range(1, 11)),
#     'bootstrap': [True, False],
# }

# # 定义网格搜索对象
# search = GridSearchCV(
#     model, param_grid=param_dist, cv=5, scoring='neg_mean_squared_error', n_jobs=4
# )

# # 进行搜索
# search.fit(train_features, train_labels)

# # 输出最优参数
# print(search.best_params_)

In [ ]:
# 创建Random Forest Regression模型
model = RandomForestRegressor(
    bootstrap=True,
    max_depth=25,
    max_features="log2",  # type: ignore
    min_samples_leaf=7,
    min_samples_split=2,
    n_estimators=55,
)

# 训练模型
model.fit(train_features, train_labels)

# 预测测试集结果
y_pred = model.predict(test_features)

# 计算MSE评分
mse = mean_squared_error(test_labels, y_pred)
r2 = r2_score(test_labels, y_pred)

# 输出MSE评分
print("MSE得分:{}".format(mse))
print("R^2得分:{}".format(r2))